## Data preparation with PANDAS

In [19]:
import pandas as pd


# Power plant conditions
p_conditions = pd.DataFrame({"Power plant":           ["Ahlen", "Fjället", "Forsen", "Kärret"],
                           "Initial reservoir level": [5800,    1000,      20,       13      ],
                           "Maximum reservoir level": [7160,    1675,      27,       13      ],
                           "Minimum reservoir level": [5800,    1000,      10,       6       ],
                           "Discharge capacity":      [540,     135,       975,      680     ],
                           "Power conversion":        [0.52,    1.17,      0.29,     0.05    ],
                           "Maximum spillage":        [820,     930,       360,      400     ],
                           "Local flow":              [177,     28,        8,        29      ],
                          })

# Time conditions
t_conditions = pd.DataFrame({"Time": range(1,10),
                             "Cost": [45, 55, 95, 80, 140, 150, 80, 70, 130]})


## GAMSPy FTW

# Initializing all our Parameters and Varaibales etc

In [20]:

from gamspy import Container, Set, Variable, Parameter, Equation, Sum, Model, Sense


m = Container()

t = Set(m, name="t", description="time in hours", records=t_conditions['Time'])
p = Set(m, name="p", description="Power plant", records=p_conditions['Power plant'])

# Parameter and Variables definitions
prices = Parameter(m, name="prices", domain=t, description="Prices (MWh) at different hours", records=t_conditions[['Time', 'Cost']])

reservoir_init = Parameter(m, name="reservoir_init", domain=p, description="Initial reservoir level", records=p_conditions[['Power plant', 'Initial reservoir level']])
reservoir_max = Parameter(m, name="reservoir_max", domain=p, description="Maximum reservoir level", records=p_conditions[['Power plant', 'Maximum reservoir level']])
reservoir_min = Parameter(m, name="reservoir_min", domain=p, description="Minimum reservoir level", records=p_conditions[['Power plant', 'Minimum reservoir level']])
discharge_max = Parameter(m, name="discharge_max", domain=p, description="Discharge capacity", records=p_conditions[['Power plant', 'Discharge capacity']])
power_conversion = Parameter(m, name="power_conversion", domain=p, description="Power conversion", records=p_conditions[['Power plant', 'Power conversion']])
spillage_max = Parameter(m, name="spillage_max", domain=p, description="Maximum spillage", records=p_conditions[['Power plant', 'Maximum spillage']])
local_flow = Parameter(m, name="local_flow", domain=p, description="Local flow", records=p_conditions[['Power plant', 'Local flow']])

discharge = Variable(m, name="discharge", type="positive", domain=[t,p], description="Discharge rate at each power plant at each time")
spillage = Variable(m, name="spillage", type="positive", domain=[t,p], description="Spillage rate at each power plant at each time")
reservoir_level = Variable(m, name="reservoir_level", type="positive", domain=[t,p], description="Reservoir level at each power plant at each time")

### Equations and condtions

In [21]:
# Discharge criteria
discharge.up[t,p] = discharge_max[p]

# Spillage criteria
spillage.up[t,p] = spillage_max[p]

# Reservoir level criteria
reservoir_level.lo[t,p] = reservoir_min[p]
reservoir_level.up[t,p] = reservoir_max[p]

# Reservoir Ahlen
res_Ahlen = Equation(m, name="res_Ahlen", domain=t, description="Reservoir level at Ahlen at different hours")
res_Ahlen[t] = reservoir_level[t-1, 'Ahlen'] + 3600*(local_flow['Ahlen'] - discharge[t,'Ahlen'] - spillage[t,'Ahlen']) == reservoir_level[t,'Ahlen']

# Reservoir Fjället
res_Fjället = Equation(m, name="res_Fjallet", domain=t, description="Reservoir level at Fjället at different hours")
res_Fjället[t] = reservoir_level[t-1, 'Fjället'] + 3600*(local_flow['Fjället'] - discharge[t,'Fjället'] - spillage[t,'Fjället']) == reservoir_level[t,'Fjället']

# Reservoir Forsen
res_Forsen = Equation(m, name="res_Forsen", domain=t, description="Reservoir level at Forsen at different hours")
res_Forsen[t] = reservoir_level[t-1, 'Forsen'] + 3600*(discharge[t-2,'Ahlen'] + spillage[t-2,'Ahlen'] + discharge[t-1,'Fjället'] + spillage[t-1,'Fjället'] + local_flow['Forsen'] - discharge[t,'Forsen'] - spillage[t,'Forsen']) == reservoir_level[t,'Forsen']

# Reservoir Kärret
res_Kärret = Equation(m, name="res_Karret", domain=t, description="Reservoir level at Kärret at different hours")
res_Kärret[t] = reservoir_level[t-1, 'Kärret'] + 3600*(discharge[t-1,'Forsen'] + spillage[t-1,'Forsen'] + local_flow['Kärret'] - discharge[t,'Kärret'] - spillage[t,'Kärret']) == reservoir_level[t,'Kärret']

### Obejctive

In [22]:
obj = Sum((t, p), prices[t]*3600*power_conversion[p]*(discharge[t,p] + spillage[t,p]))

In [23]:
flow = Model(m, name="flow", equations=m.getEquations(), objective=obj, problem="LP", sense=Sense.MAX)
flow.solve(solver='CPLEX')

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,577623696.3,37,109,LP,CPLEX,0.003


In [26]:
discharge.pivot()

,Ahlen,Fjället,Forsen,Kärret
1,175.011111,27.534722,7.992500,0.000000
2,0.000000,28.000000,0.000000,36.992500
3,177.377778,0.000000,211.015833,0.000000
4,176.622222,28.000000,212.995278,240.013889
5,0.000000,28.187500,213.382500,241.995278
6,0.000000,0.000000,0.000000,242.384444
7,0.000000,28.000000,212.995278,241.809722
8,176.622222,27.812500,213.377778,0.000000
9,0.000000,28.187500,0.000000,242.379722


In [25]:
spillage.pivot()

,Ahlen,Fjället,Forsen,Kärret
1,0.000000,0.0,0.000000,28.996389
2,177.000000,0.0,35.534722,0.000000
3,0.000000,28.0,0.000000,64.536667
4,0.000000,0.0,0.000000,0.000000
5,177.000000,0.0,0.000000,0.000000
6,177.377778,28.0,212.809722,0.000000
7,177.000000,0.0,0.000000,0.000000
8,0.000000,0.0,0.000000,241.993333
9,177.377778,0.0,212.817222,0.000000
